<a href="https://colab.research.google.com/github/EvgenyBelov78/DA1/blob/main/ML_HW_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import datetime

In [ ]:
import urllib.request
url = "http://www.cbr.ru/scripts/XML_dynamic.asp?date_req1=02/03/2001&date_req2=04/10/2022&VAL_NM_RQ=R01235"

In [ ]:
df = pd.read_xml(url)

In [ ]:
df.tail()

,Date,Id,Nominal,Value
5348,28.09.2022,R01235,1,"58,1756"
5349,29.09.2022,R01235,1,"58,4485"
5350,30.09.2022,R01235,1,"57,4130"
5351,01.10.2022,R01235,1,"55,2987"
5352,04.10.2022,R01235,1,"57,5664"


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5353 entries, 0 to 5352
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Date     5353 non-null   object
 1   Id       5353 non-null   object
 2   Nominal  5353 non-null   int64 
 3   Value    5353 non-null   object
dtypes: int64(1), object(3)
memory usage: 167.4+ KB


In [ ]:
pd.to_datetime(df['Date'])

0      2002-01-01
1      2003-01-01
2      2004-01-01
3      2005-01-01
4      2009-01-01
          ...    
5348   2015-12-31
5349   2016-12-31
5350   2019-12-31
5351   2020-12-31
5352   2021-12-31
Name: Date, Length: 5353, dtype: datetime64[ns]

In [ ]:
df.drop ('Id',axis=1,inplace=True)

In [ ]:
df.drop ('Nominal',axis=1,inplace=True)

In [ ]:
df = df.apply(lambda x: x.str.replace(',','.'))

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5353 entries, 0 to 5352
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    5353 non-null   object
 1   Value   5353 non-null   object
dtypes: object(2)
memory usage: 83.8+ KB


In [ ]:
df = df.sort_values(by='Date').reset_index(drop=True)

In [ ]:
import matplotlib.pyplot as plt

df.set_index('Date').plot()

TypeError: ignored

In [ ]:
df = df.groupby('dDate')['nCur'].mean().reset_index()

In [ ]:
df.set_index('Date').plot()

TypeError: ignored

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df.set_index('dDate', inplace = True)

In [ ]:
df

,nCur
dDate,
2021-08-03,72.93570
2021-08-04,73.09355
2021-08-05,73.09740
2021-08-06,73.31135
2021-08-09,73.55665
...,...
2022-09-27,58.25000
2022-09-28,57.89825
2022-09-29,57.31995


In [ ]:
# функция по созданию сгенерированных признаков
def make_features(data, max_lag, rolling_mean_size):
    data['year'] = data.index.year
    data['month'] = data.index.month
    data['day'] = data.index.day
    data['dayofweek'] = data.index.dayofweek
    data['is_weekend'] = data.index.isin([5,6])*1
    
    for lag in range(1, max_lag + 1):
        data['lag_{}'.format(lag)] = data['nCur'].shift(lag)

    data['y_mean'] = data['nCur'].shift().rolling(rolling_mean_size).mean().copy()

In [ ]:
make_features(df,30,3)

In [ ]:
df

,nCur,year,month,day,dayofweek,is_weekend,lag_1,lag_2,lag_3,lag_4,...,lag_22,lag_23,lag_24,lag_25,lag_26,lag_27,lag_28,lag_29,lag_30,y_mean
dDate,,,,,,,,,,,,,,,,,,,,,
2021-08-03,72.93570,2021,8,3,1,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-08-04,73.09355,2021,8,4,2,0,72.93570,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-08-05,73.09740,2021,8,5,3,0,73.09355,72.93570,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-08-06,73.31135,2021,8,6,4,0,73.09740,73.09355,72.93570,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.042217
2021-08-09,73.55665,2021,8,9,0,0,73.31135,73.09740,73.09355,72.93570,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.167433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-27,58.25000,2022,9,27,1,0,58.11560,57.68065,59.20265,60.63150,...,60.26615,59.76960,59.90720,59.85330,59.63390,59.04825,59.47380,60.64310,61.10670,58.332967
2022-09-28,57.89825,2022,9,28,2,0,58.25000,58.11560,57.68065,59.20265,...,60.31390,60.26615,59.76960,59.90720,59.85330,59.63390,59.04825,59.47380,60.64310,58.015417
2022-09-29,57.31995,2022,9,29,3,0,57.89825,58.25000,58.11560,57.68065,...,60.15750,60.31390,60.26615,59.76960,59.90720,59.85330,59.63390,59.04825,59.47380,58.087950


In [ ]:
df.dropna(inplace = True)

In [ ]:
df

,nCur,year,month,day,dayofweek,is_weekend,lag_1,lag_2,lag_3,lag_4,...,lag_22,lag_23,lag_24,lag_25,lag_26,lag_27,lag_28,lag_29,lag_30,y_mean
dDate,,,,,,,,,,,,,,,,,,,,,
2021-09-14,72.86035,2021,9,14,1,0,72.85360,72.95585,72.95395,73.35640,...,73.33035,73.52655,73.91075,73.76995,73.55665,73.31135,73.09740,73.09355,72.93570,72.921133
2021-09-15,72.57490,2021,9,15,2,0,72.86035,72.85360,72.95585,72.95395,...,73.34000,73.33035,73.52655,73.91075,73.76995,73.55665,73.31135,73.09740,73.09355,72.889933
2021-09-16,72.52145,2021,9,16,3,0,72.57490,72.86035,72.85360,72.95585,...,73.49575,73.34000,73.33035,73.52655,73.91075,73.76995,73.55665,73.31135,73.09740,72.762950
2021-09-17,72.72745,2021,9,17,4,0,72.52145,72.57490,72.86035,72.85360,...,73.74725,73.49575,73.34000,73.33035,73.52655,73.91075,73.76995,73.55665,73.31135,72.652233
2021-09-20,73.43670,2021,9,20,0,0,72.72745,72.52145,72.57490,72.86035,...,74.19740,73.74725,73.49575,73.34000,73.33035,73.52655,73.91075,73.76995,73.55665,72.607933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-27,58.25000,2022,9,27,1,0,58.11560,57.68065,59.20265,60.63150,...,60.26615,59.76960,59.90720,59.85330,59.63390,59.04825,59.47380,60.64310,61.10670,58.332967
2022-09-28,57.89825,2022,9,28,2,0,58.25000,58.11560,57.68065,59.20265,...,60.31390,60.26615,59.76960,59.90720,59.85330,59.63390,59.04825,59.47380,60.64310,58.015417
2022-09-29,57.31995,2022,9,29,3,0,57.89825,58.25000,58.11560,57.68065,...,60.15750,60.31390,60.26615,59.76960,59.90720,59.85330,59.63390,59.04825,59.47380,58.087950


In [ ]:
X_train,X_test, y_train, y_test = train_test_split(df.drop('nCur',axis=1),
                                                  df.nCur,
                                                  shuffle = False,
                                                  test_size = 0.19)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(X_train, y_train)

LinearRegression()

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
print('Ошибка на Train',round(mean_absolute_error(y_train, lr.predict(X_train)),2))

Ошибка на Train 1.03


In [ ]:
print('Ошибка на Test',round(mean_absolute_error(y_test, lr.predict(X_test)),2))

Ошибка на Test 0.52


## Предсказываем на 4 октября

In [ ]:
df_new = pd.DataFrame(df['nCur']).reset_index().copy()

In [ ]:
df_new

,dDate,nCur
0,2021-09-14,72.86035
1,2021-09-15,72.57490
2,2021-09-16,72.52145
3,2021-09-17,72.72745
4,2021-09-20,73.43670
...,...,...
261,2022-09-27,58.25000
262,2022-09-28,57.89825
263,2022-09-29,57.31995
264,2022-09-30,56.42260


In [ ]:
df_concat = df_new.tail(1).copy()

In [ ]:
df_concat.reset_index( inplace = True , drop=True)

In [ ]:
df_concat

,dDate,nCur
0,2022-10-03,57.3813


In [ ]:
df_concat['dDate'] = pd.to_datetime('2022-10-04')

In [ ]:
import numpy as np
df_concat['nCur'] = 0

In [ ]:
df_concat

,dDate,nCur
0,2022-10-04,0


In [ ]:
df2 = pd.concat([df_new,df_concat], ignore_index=True)

### генерируем фичи для 4 октября

In [ ]:
df2.set_index('dDate', inplace = True)

In [ ]:
df2.tail()

,nCur
dDate,
2022-09-28,57.89825
2022-09-29,57.31995
2022-09-30,56.42260
2022-10-03,57.38130
2022-10-04,0.00000


In [ ]:
make_features(df2,30,3)

In [ ]:
df2.drop('nCur',axis=1).tail(1)

,year,month,day,dayofweek,is_weekend,lag_1,lag_2,lag_3,lag_4,lag_5,...,lag_22,lag_23,lag_24,lag_25,lag_26,lag_27,lag_28,lag_29,lag_30,y_mean
dDate,,,,,,,,,,,,,,,,,,,,,
2022-10-04,2022,10,4,1,0,57.3813,56.4226,57.31995,57.89825,58.25,...,60.4812,60.2526,60.17585,60.1575,60.3139,60.26615,59.7696,59.9072,59.8533,57.041283


In [ ]:
lr_full = LinearRegression()

In [ ]:
lr_full.fit(df.drop('nCur',axis=1),df.nCur)

LinearRegression()

In [ ]:
print('Предсказание на 4 октября - ',lr.predict(df2.drop('nCur',axis=1).tail(1))[0])

Предсказание на 4 октября -  56.699756286337106


In [ ]:
print('Предсказание на 4 октября, модель видела весь датасет - ',lr_full.predict(df2.drop('nCur',axis=1).tail(1))[0])

Предсказание на 4 октября, модель видела весь датасет -  56.77565174459596


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor

In [ ]:
gb = GradientBoostingRegressor()

In [ ]:
gb.fit(df.drop('nCur',axis=1),df.nCur)

GradientBoostingRegressor()

In [ ]:
gb.predict(df2.drop('nCur',axis=1).tail(1))[0]

58.41623427546648

57,5664 ₽  курс 4 октября